<a href="https://colab.research.google.com/github/dominiksakic/zero_to_hero/blob/main/adv_05_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal
Train a GPT to do addition of two numbers, i.e. a+b=c. You may find it helpful to predict the digits of c in reverse order, as the typical addition algorithm (that you're hoping it learns) would proceed right to left too. You may want to modify the data loader to simply serve random problems and skip the generation of train.bin, val.bin. You may want to mask out the loss at the input positions of a+b that just specify the problem using y=-1 in the targets (see CrossEntropyLoss ignore_index). Does your Transformer learn to add? Once you have this, swole doge project: build a calculator clone in GPT, for all of +-*/. Not an easy problem. You may need Chain of Thought traces.


## research and refresher

- Simple example:
  - 1 + 9 = 01
  - then tokenize it:     1, 10, 9, 11, 0 , 1
  - then create targets: -1, -1, -1, -1, 0, 1
  - using ignore_index -1 will not calculate the crossentropy for  the index that match -1 in the targets.


### Reminder of cross entropy:
- [2.0, 1.0, 0.1]
- Target 2

- exp(2.0) = 7.3898
- exp(1.0) = 2.7128
- exp(0.1) = 1.105

- sum = 10.212

- 7.3898/10 = 0.7235
- 2.7128/10 = 0.206
- 1.105/10 = 0.108

- target is 2 -> 0.108
- -log(0.108) => 2.2256


## Think about what are the targets of the inputs?
- For gpt each token and the rest of the block size will be used to predict the next token.

In [63]:
itos = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '=']
stoi = {ch:i for i, ch in enumerate(itos)}
vocab_size = len(itos)

def encode(s):
  return [stoi[ch] for ch in s]

def decode(tokens):
  return ''.join([itos[t] for t in tokens])

print(encode("1+9=01"))  # [1, 10, 9, 11, 0, 1]
print(decode([1, 10, 9, 11, 0, 1]))  # '1+9=01'

[1, 10, 9, 11, 0, 1]
1+9=01


In [64]:
import random
import torch
from typing import List, Tuple
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def make_problem(max_digits : int =2, reverse_answer: bool=True) -> List:
    a = random.randint(0, 10**max_digits - 1)
    b = random.randint(0, 10**max_digits - 1)
    c = a + b
    answer = str(c)

    if reverse_answer:
        answer = answer[::-1]

    problem_str = f"{a}+{b}={answer}"
    x = encode(problem_str)
    y = x[1:] + [ -1 ]  # mask problem part

    return x, y

# test:
x, y = make_problem()
print("x:", x)
print("y:", y)
print("decoded x:", decode(x))

x: [5, 0, 10, 5, 2, 11, 2, 0, 1]
y: [0, 10, 5, 2, 11, 2, 0, 1, -1]
decoded x: 50+52=201


In [65]:
def make_batch(
      batch_size: int,
      max_digits : int =2,
      reverse_answer : bool =True
    ) -> Tuple[torch.Tensor, torch.Tensor]:

    xs, ys = [], []
    for _ in range(batch_size):
        x, y = make_problem(max_digits, reverse_answer)
        xs.append(x)
        ys.append(y)

    max_len = max(len(seq) for seq in xs)
    for i in range(batch_size):
        pad_len = max_len - len(xs[i])
        xs[i] = xs[i] + [0] * pad_len
        ys[i] = ys[i] + [-1] * pad_len

    xs = torch.tensor(xs)
    ys = torch.tensor(ys)
    xs, ys = xs.to(device), ys.to(device)
    return xs, ys

# test:
xb, yb = make_batch(4)
print(xb)
print(yb)

tensor([[ 1,  1, 10,  2,  2, 11,  3,  3,  0],
        [ 8,  6, 10,  8,  6, 11,  2,  7,  1],
        [ 8,  2, 10,  3,  8, 11,  0,  2,  1],
        [ 4,  8, 10,  1,  4, 11,  2,  6,  0]], device='cuda:0')
tensor([[ 1, 10,  2,  2, 11,  3,  3, -1, -1],
        [ 6, 10,  8,  6, 11,  2,  7,  1, -1],
        [ 2, 10,  3,  8, 11,  0,  2,  1, -1],
        [ 8, 10,  1,  4, 11,  2,  6, -1, -1]], device='cuda:0')


In [66]:
import torch
max_digits = 2
batch_size = 128
block_size = 3 * max_digits + 3
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0

In [67]:
torch.manual_seed(1337)

In [68]:
def apply_rope(q, k):
    # q, k: (B, n_head, T, head_size), where head_size must be even
    B, nh, T, hs = q.shape
    assert hs % 2 == 0, "head_size must be even for RoPE"
    half = hs // 2

    freqs = torch.exp(-torch.arange(0, half, dtype=torch.float32) * math.log(10000) / half).to(q.device)  # (half,)
    positions = torch.arange(T, device=q.device).float()  # (T,)
    angles = torch.einsum('t,d->td', positions, freqs)  # (T, half)
    sin = angles.sin().unsqueeze(0).unsqueeze(0)  # (1, 1, T, half)
    cos = angles.cos().unsqueeze(0).unsqueeze(0)  # (1, 1, T, half)

    q1, q2 = q[..., :half], q[..., half:]
    k1, k2 = k[..., :half], k[..., half:]
    q_rotated = torch.cat([q1 * cos - q2 * sin, q1 * sin + q2 * cos], dim=-1)
    k_rotated = torch.cat([k1 * cos - k2 * sin, k1 * sin + k2 * cos], dim=-1)
    return q_rotated, k_rotated

In [69]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    losses = []
    for _ in range(eval_iters):
        X, Y = make_batch(batch_size, max_digits=2, reverse_answer=True)
        _, l = model(X, Y)
        losses.append(l.item())
    model.train()
    return sum(losses) / len(losses)

In [70]:
from torch import nn
from torch.nn import functional as F
import math

class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel."""

    def __init__(self, n_embd, num_heads, dropout):
        super().__init__()
        assert n_embd % num_heads == 0
        self.n_head = num_heads
        self.head_size = n_embd // num_heads
        self.dropout = nn.Dropout(dropout)

        # Linear projections for q, k, v (combined head projection)
        self.key = nn.Linear(n_embd, n_embd, bias=False)
        self.query = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(n_embd, n_embd, bias=False)

        # Final projection layer
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        B, T, C = x.shape  # (batch, time, channels)

        # Project and reshape into multiple heads
        k = self.key(x).view(B, T, self.n_head, self.head_size).transpose(1, 2)  # (B, nh, T, hs)
        q = self.query(x).view(B, T, self.n_head, self.head_size).transpose(1, 2)  # (B, nh, T, hs)
        v = self.value(x).view(B, T, self.n_head, self.head_size).transpose(1, 2)  # (B, nh, T, hs)

        # Apply Rotary Positional Embedding if available
        q, k = apply_rope(q, k)

        # Compute attention weights
        wei = q @ k.transpose(-2, -1) * (self.head_size ** -0.5)  # (B, nh, T, T)

        # Mask to prevent attending to future tokens (causal attention)
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)  # (1, 1, T, T)
        wei = wei.masked_fill(mask == 0, float('-inf'))

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # Weighted sum of values
        out = wei @ v  # (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B, T, C)  # reassemble heads (B, T, C)
        out = self.dropout(self.proj(out))
        return out

In [71]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_embd, n_head, dropout)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [72]:
class GPTLanguageModelRoPE(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        x = self.blocks(tok_emb) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=-1)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, _ = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModelRoPE()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.794892 M parameters


In [73]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        loss = estimate_loss()
        print(f"step {iter}: random generated validation loss {loss:.4f}")

    # sample a batch of data
    xb, yb = make_batch(batch_size=batch_size, max_digits=max_digits)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: random generated validation loss 2.4967
step 100: random generated validation loss 1.6226
step 200: random generated validation loss 1.6133
step 300: random generated validation loss 1.5734
step 400: random generated validation loss 1.1466
step 500: random generated validation loss 1.0026
step 600: random generated validation loss 0.9925
step 700: random generated validation loss 0.9921
step 800: random generated validation loss 0.9918
step 900: random generated validation loss 0.9886
step 1000: random generated validation loss 1.0065
step 1100: random generated validation loss 0.9914
step 1200: random generated validation loss 0.9931
step 1300: random generated validation loss 0.9879
step 1400: random generated validation loss 0.9894
step 1500: random generated validation loss 0.9873
step 1600: random generated validation loss 0.9866
step 1700: random generated validation loss 0.9857
step 1800: random generated validation loss 0.9865
step 1900: random generated validation loss

In [79]:
# generate from the model
inp = torch.tensor([encode("3+7=")], dtype=torch.long, device=device)
out = model.generate(inp, max_new_tokens=2)
print("decoded:", decode(out[0].tolist()))

decoded: 3+7=01
